In [1]:
import xarray as xr
import xesmf as xe
import numpy as np
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
def flatten_list(input_list):
    flattened_list = []
    for item in input_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list

FNS_all_olr = []
for yryr in range(1974,2023):
    for momo in range(1,13):
        FNS = sorted(glob.glob('/glade/collections/rda/data/ds633.0//e5.oper.an.sfc/'+str(yryr)+str(momo).zfill(2)+'/*_sstk.*.nc'))
        FNS_all_olr.append(FNS)
FNS_all_olr = flatten_list(FNS_all_olr)
# FNS_all_olr

In [1]:
FNS_all_olr
dirout = '/glade/scratch/wchapman/ERA5_uvolr/'
for ee,fnfn in enumerate(FNS_all_olr):

    DSall = xr.open_dataset(fnfn)    
    inits=[]
    for initt in DSall['time'].values:
        inits.append(str(initt)[:10])
    
    inits = np.unique(inits)
    for inny in inits: 
        file_out_name = dirout + fnfn.split('/')[-1].split('ll025sc')[0]+'ll025sc.'+inny.replace('-','')+'.nc'
        print(file_out_name)
        if os.path.exists(file_out_name):
            print(f"File '{file_out_name}' already exists. Continuing to the next file...")
            continue
            
        DSsmall = DSall.sel(time=inny)
        
        DSsmall = DSsmall.mean(['time'])
        dtrang=pd.date_range(start=str(inny),end='2024-01-01')[0]
        DSsmall = DSsmall.expand_dims(time=[dtrang]).drop('utc_date')
        DSsmall.to_netcdf(file_out_name)


## Regrid to 1deg

In [3]:
FNSall = sorted(glob.glob('/glade/scratch/wchapman/ERA5_uvolr/*_sstk*.nc'))
#grab an example file to get the grid: 
DScamgrid = xr.open_dataset('/glade/campaign/cisl/aiml/wchapman/CAM_runs/f.e21.DAcompset.f09_d025_free_MJO_1982/atm/hist/f.e21.DAcompset.f09_d025_free_MJO_1982.cam.h0.1982-02.nc')
DScamgrid

#make regridder structure:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.array(DScamgrid.lat)),
        "lon": (["lon"], np.array(DScamgrid.lon)),
    }
)

#gather dates
fnunique = np.unique([fnfn.split('.')[-2] for fnfn in FNSall])

fnunique[14850]

'20140829'

In [1]:
FNSall = sorted(glob.glob('/glade/scratch/wchapman/ERA5_uvolr/*_sstk*.nc'))
#grab an example file to get the grid: 
DScamgrid = xr.open_dataset('/glade/campaign/cisl/aiml/wchapman/CAM_runs/f.e21.DAcompset.f09_d025_free_MJO_1982/atm/hist/f.e21.DAcompset.f09_d025_free_MJO_1982.cam.h0.1982-02.nc')
DScamgrid

#make regridder structure:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.array(DScamgrid.lat)),
        "lon": (["lon"], np.array(DScamgrid.lon)),
    }
)

#gather dates
fnunique = np.unique([fnfn.split('.')[-2] for fnfn in FNSall])




print('starting loop')
for ee,dtdt in enumerate(fnunique[14850:]):
    ee+=1
    if ee%50==0:
        print(f'loop {ee} of {len(fnunique)}')
    FNSdtdt = sorted(glob.glob('/glade/scratch/wchapman/ERA5_uvolr/*_sstk*'+dtdt+'*.nc'))
    
    if len(FNSdtdt) !=1:
        print('stopped at:', dtdt)
        break
        
    for indfiles in FNSdtdt: 
        #do the rest of my operations
            
        outfile = '/glade/scratch/wchapman/ERA5_uvolr/1deg/'+indfiles.split('/')[-1][:-2]+'1deg.nc'
        if os.path.exists(outfile):
            # File exists, continue with the loop iteration
            print(f"File '{outfile}' exists. Continuing with the loop iteration...")
            # Additional actions for the existing file...
            continue  # Continue to the next iteration
            
        DSall = xr.open_dataset(indfiles)
        # ##build regridder method: 
        if ee == 0:
            print('...rebuilding weight file...')
            regridder_save_weights = xe.Regridder(DSall, ds_out, "bilinear",filename='ECMWF_to_1deg_xesmf_712x1440_180x360.nc')
            # #regrid. 
            ds_out = regridder_save_weights(DSall) #this is an xarray instance now. 
            #compute from xarray in dask 
            #save:
            print('saving... ',outfile)
            ds_out.to_netcdf(outfile)
        else:
            regridder_reuse_weights = xe.Regridder(DSall, ds_out, "bilinear",reuse_weights=True,filename='ECMWF_to_1deg_xesmf_712x1440_180x360.nc')
            # #regrid. 
            ds_out = regridder_reuse_weights(DSall) #this is an xarray instance now. 
            #compute from xarray in dask 
            #save:
            print('saving... ',outfile)
            ds_out.to_netcdf(outfile)




In [5]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

client does not exist yet
